## WFP-01-01-02 Sentinel-1 coherence timeseries

This application takes a pair of Sentinel-1 products and identifies and generates the coherence

### <a name="quicklink">Quick link

* [Objective](#objective)
* [Test Site](#test-site)
* [Context](#context)
* [Applicability](#applicability)
* [Data](#data)
* [Service Definition](#service)
* [Parameter Definition](#parameter)
* [Runtime Parameter Definition](#runtime)
* [Workflow](#workflow)
* [Strengths and Limitations](#strengths-limitations) 
* [License](#license)

As a data processor developer, I want to implement, and package an algorithm processing a pair of S1 SAR SLC datasets using the SNAP toolbox notebook archetype with the following processing steps:

A. S1 SAR processing per image:

* Application of orbit file (should wait for the orbit file a couple of days, since for coherence this is important)
* TOPS slice assembly (if necessary)
* TOPS split (if necessary)

B. For image pair:

* TOPS coregistration
* Coherence estimation (Given set of images from same orbit,  {t1, t2, t3, ... , tn}, two temporally adjacent images would * constitute an image pair, with the first one being the master. So the pairs would be: {t1 - t2, t2 - t3, ... , tn-1 - tn}.)
* TOPS deburst
* Multi-looking
* Terrain correction


### <a name="service">Service definition

In [1]:
service = dict([('title', 'WFP-01-01-02 Sentinel-1 coherence'),
                ('abstract', 'WFP-01-01-02 Sentinel-1 coherence'),
                ('id', 'ewf-wfp-01-01-02')])

In [2]:
swaths = dict([('id', 'swaths'),
               ('value', 'IW1,IW2,IW3'),
               ('title', 'Sentinel-1 sub-swaths'),
               ('abstract', 'Sentinel-1 sub-swaths')])

In [3]:
subswaths = list(swaths['value'].split(',')) 

In [4]:
polarisation = dict([('id', 'polarisation'),
                     ('value', 'VH'),
                     ('title', 'Selected Polarisation'),
                     ('abstract', 'Choose ''VV''or ''VH'' or ''VV,VH'' to include all Polarisations')])

In [5]:
polarisations = list(polarisation['value'].split(',')) 

In [6]:
if len(polarisations)==2:
    selected_Pol=False 
else:
    selected_Pol=True

### <a name="runtime">Runtime parameter definition

**Input identifiers**

These are the Sentinel-1 product identifiers

In [7]:
input_identifiers = ('S1A_IW_SLC__1SDV_20170105T035027_20170105T035041_014691_017E74_5F66',
                     'S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110',
                     'S1A_IW_SLC__1SDV_20170117T035008_20170117T035037_014866_0183DD_C2F1')

**Input references**

These are the Sentinel-1 catalogue references

In [8]:
input_references = ('https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20170105T035027_20170105T035041_014691_017E74_5F66',
                    'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20170105T035000_20170105T035030_014691_017E74_0110',
                    'https://catalog.terradue.com/sentinel1/search?format=atom&uid=S1A_IW_SLC__1SDV_20170117T035008_20170117T035037_014866_0183DD_C2F1')

**Data path**

This path defines where the data is staged-in. 

In [9]:
data_path = '/workspace/data'

### <a name="workflow">Workflow

#### Import the packages required for processing the data

In [ ]:
from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import dateutil.parser as parser
import gc
from datetime import datetime

import gzip
import shutil

import gdal
import osr

import lxml.etree as etree

from shapely.wkt import loads
import numpy as np

from shapely.geometry import box

import warnings
warnings.filterwarnings("ignore")
import os
import sys
import glob
sys.path.append('/opt/anaconda/bin/')

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from snappy import jpy
from snappy import ProductIO
from snappy import GPF
from snappy import HashMap

import gc
import cioppy

import lxml.etree as etree

from shapely.wkt import loads

import gdal
import osr

from shapely.geometry import box

import gzip
import shutil

#### Read the products

In [14]:
s1meta = "manifest.safe"

slave_date = ''

slave_products = []
master_products = []

slave_prefix = []
master_prefix = []

dates = []

for index, identifier in enumerate(input_identifiers):

    s1_zip_file = os.path.join(data_path, identifier + '.zip') 
    s1_meta_file = os.path.join(data_path, identifier, identifier + '.SAFE', 'manifest.safe') 

    if os.path.isfile(s1_zip_file):
        s1prd = s1_zip_file
    elif os.path.isfile(s1_meta_file):
        s1prd = s1_meta_file

    reader = ProductIO.getProductReader("SENTINEL-1")
    product = reader.readProductNodes(s1prd, None)
    
    
    width = product.getSceneRasterWidth()
    height = product.getSceneRasterHeight()
    name = product.getName()
    start_date = parser.parse(product.getStartTime().toString()).isoformat()
    
    dates.append(start_date[:19])
    if index == 0:
        
        slave_date = start_date[:10]
        slave_products.append(product)
        print("Product: %s, %d x %d pixels of %s assigned as slave" % (name, width, height, start_date))
        
        slave_data_take = identifier.split('_')[-2]  
        slave_prefix.append(identifier.split('_')[-1]) 
    else:
        
        if start_date[:10] == slave_date:
            slave_products.append(product)
            print("Product: %s, %d x %d pixels of %s assigned as slave" % (name, width, height, start_date))
            slave_prefix.append(identifier.split('_')[-1]) 
        else:
            master_products.append(product)
            print("Product: %s, %d x %d pixels of %s assigned as master" % (name, width, height, start_date))
            master_data_take = identifier.split('_')[-2]  
            master_prefix.append(identifier.split('_')[-1]) 

            
            
output_name = 'S1_COH_%s_%s_%s_%s_%s_%s_%s_%s_%s' % (parser.parse(min(dates)).strftime('%Y%m%d%H%M%S'),
                                                     parser.parse(max(dates)).strftime('%Y%m%d%H%M%S'),
                                                     slave_data_take,
                                                     len(input_identifiers)/2, 
                                                     '_'.join(slave_prefix),
                                                     master_data_take,
                                                     len(input_identifiers)/2, 
                                                     '_'.join(master_prefix),
                                                    polarisation['value'])



NameError: name 'input_identifiers' is not defined

In [11]:
output_name

'S1_COH_20170105035000_20170117035008_017E74_1_5F66_0110_0183DD_1_C2F1'

#### Assemble the slices

In [12]:
operator = 'SliceAssembly'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('selectedPolarisations', None)


In [13]:
if len(slave_products) > 1:
    parameters = HashMap()
    if selected_Pol:
        parameters.put('selectedPolarisations', polarisation['value'])
    else:
        parameters.put('selectedPolarisations', None)
    slave_slice = GPF.createProduct(operator,parameters, slave_products)

else:
    slave_slice = slave_products[0]

In [14]:
if len(master_products) >1:

    parameters = HashMap()
    if selected_Pol:
        parameters.put('selectedPolarisations', polarisation['value'])
    else:
        parameters.put('selectedPolarisations', None)
    

    master_slice = GPF.createProduct(operator,
                       parameters, 
                       master_products)

else: 
    master_slice = master_products[0] 

### Apply orbit file

In [15]:
operator = 'Apply-Orbit-File'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('orbitType', 'Sentinel Precise (Auto Download)')
('polyDegree', '3')
('continueOnFail', 'false')


In [16]:
master_orbit_products = []
slave_orbit_products = []

HashMap = jpy.get_type('java.util.HashMap')
GPF.getDefaultInstance().getOperatorSpiRegistry().loadOperatorSpis()

parameters = HashMap()

parameters.put('orbitType', 'Sentinel Precise (Auto Download)')
parameters.put('polyDegree', '3')
parameters.put('continueOnFail', 'false')


master_orbit = GPF.createProduct(operator,
                                 parameters, 
                                 master_slice)
    

slave_orbit= GPF.createProduct(operator,
                               parameters, 
                               slave_slice)
  

### Split

In [17]:
operator = 'TOPSAR-Split'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())


('subswath', None)
('selectedPolarisations', None)
('firstBurstIndex', '1')
('lastBurstIndex', '9999')
('wktAoi', None)


In [18]:
master_split_prds = []
slave_split_prds= []

for subswath in subswaths:  
    
    parameters = HashMap()

    parameters.put('subswath', subswath)
    if selected_Pol:
        parameters.put('selectedPolarisations', polarisation['value'])
    else:
        parameters.put('selectedPolarisations', None)
    
    parameters.put('firstBurstIndex', '1')
    parameters.put('lastBurstIndex', '9999')

    master_split_prds.append(GPF.createProduct(operator,
                           parameters, 
                           master_orbit))   
    
    slave_split_prds.append(GPF.createProduct(operator,
                           parameters, 
                           slave_orbit))   
    


In [19]:
#master_orbit = None
#slave_orbit = None
#gc.collect() 

### Back-Geocoding

In [20]:
operator = 'Back-Geocoding'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('demName', 'SRTM 3Sec')
('demResamplingMethod', 'BICUBIC_INTERPOLATION')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('resamplingType', 'BISINC_5_POINT_INTERPOLATION')
('maskOutAreaWithoutElevation', 'true')
('outputRangeAzimuthOffset', 'false')
('outputDerampDemodPhase', 'false')
('disableReramp', 'false')


In [21]:

backgeo_prds = []

for index, subswath in enumerate(subswaths):  
    

    parameters = HashMap()

    for param in op_params:
        parameters.put(param.getName(), param.getDefaultValue())


    backgeo_prds.append(GPF.createProduct(operator,
                           parameters, 
                           [master_split_prds[index], 
                            slave_split_prds[index]
                           ]))


In [22]:
gc.collect()

0

In [23]:
operator = 'Coherence'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

parameters = HashMap()

for param in op_params:
    print(param.getName(), param.getDefaultValue())
    parameters.put(param.getName(), param.getDefaultValue())


('cohWinAz', '10')
('cohWinRg', '10')
('subtractFlatEarthPhase', 'false')
('srpPolynomialDegree', '5')
('srpNumberPoints', '501')
('orbitDegree', '3')
('squarePixel', 'true')
('subtractTopographicPhase', 'false')
('demName', 'SRTM 3Sec')
('externalDEMFile', None)
('externalDEMNoDataValue', '0')
('externalDEMApplyEGM', 'true')
('tileExtensionPercent', '100')


In [24]:
coherence_prds = []

for index, subswath in enumerate(subswaths):  
    

    parameters = HashMap()

    for param in op_params:
        parameters.put(param.getName(), param.getDefaultValue())

    coherence_prds.append(GPF.createProduct(operator,
                           parameters, 
                           backgeo_prds[index]))



#back_geocoding_iw1 = None
#back_geocoding_iw2 = None
#back_geocoding_iw3 = None

In [25]:
gc.collect()

0

In [26]:
operator = 'TOPSAR-Deburst'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())


('selectedPolarisations', None)


In [27]:
deburst_prds = []

for index, subswath in enumerate(subswaths):  
 
    parameters = HashMap()
    if selected_Pol:
        parameters.put('selectedPolarisations', polarisation['value'])
    else:
        parameters.put('selectedPolarisations', None)
    

    deburst_prds.append(GPF.createProduct(operator,
                           parameters, 
                           coherence_prds[index]))


In [28]:
#coherence_iw1 = None
#coherence_iw2 = None
#coherence_iw3 = None

#gc.collect()

In [29]:
operator = 'TOPSAR-Merge'

op_spi = GPF.getDefaultInstance().getOperatorSpiRegistry().getOperatorSpi(operator)

op_params = op_spi.getOperatorDescriptor().getParameterDescriptors()

for param in op_params:
    print(param.getName(), param.getDefaultValue())

('selectedPolarisations', None)


In [30]:
parameters = HashMap()
if selected_Pol:
        parameters.put('selectedPolarisations', polarisation['value'])
else:
        parameters.put('selectedPolarisations', None)


tops_merge = GPF.createProduct(operator,
                           parameters, 
                           deburst_prds)



In [31]:
#deburst_iw1 = None
#deburst_iw2 = None
#deburst_iw3 = None

#gc.collect()


In [32]:
band_name = list(tops_merge.getBandNames())[0]

In [33]:
parameters = HashMap()

parameters.put('sourceBands', band_name)
parameters.put('demName', 'SRTM 1Sec HGT')
parameters.put('externalDEMFile', '')
parameters.put('externalDEMNoDataValue', '0.0')
parameters.put('externalDEMApplyEGM', 'true')
parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('imgResamplingMethod', 'BILINEAR_INTERPOLATION')
parameters.put('pixelSpacingInMeter', '10.0')
#parameters.put('pixelSpacingInDegree', '8.983152841195215E-5')
parameters.put('mapProjection', 'AUTO:42001')
parameters.put('nodataValueAtSea', 'true')
parameters.put('saveDEM', 'false')
parameters.put('saveLatLon', 'false')
parameters.put('saveIncidenceAngleFromEllipsoid', 'false')
parameters.put('saveProjectedLocalIncidenceAngle', 'false')
parameters.put('saveSelectedSourceBand', 'true')
parameters.put('outputComplex', 'false')
parameters.put('applyRadiometricNormalization', 'false')
parameters.put('saveSigmaNought', 'false')
parameters.put('saveGammaNought', 'false')
parameters.put('saveBetaNought', 'false')
parameters.put('incidenceAngleForSigma0', 'Use projected local incidence angle from DEM')
parameters.put('incidenceAngleForGamma0', 'Use projected local incidence angle from DEM')
parameters.put('auxFile', 'Latest Auxiliary File')

terrain_correction = GPF.createProduct('Terrain-Correction', parameters, tops_merge)

In [34]:
#tops_merge = None
#gc.collect()

In [35]:
#terrain_correction = None
#gc.collect()



In [ ]:
ProductIO.writeProduct(terrain_correction, output_name + '.tif', 'GeoTIFF-BigTIFF')

#### Results metadata

In [ ]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % sar)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)

    polarisation_channels = etree.SubElement(acquisition, '{%s}polarisationChannels' % eop)
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

#### Get the result WKT


In [ ]:
src = gdal.Open(output_name + '.tif')
ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

max_x = ulx + (src.RasterXSize * xres)
min_y = uly + (src.RasterYSize * yres)
min_x = ulx 
max_y = uly

source = osr.SpatialReference()
source.ImportFromWkt(src.GetProjection())

target = osr.SpatialReference()
target.ImportFromEPSG(4326)

transform = osr.CoordinateTransformation(source, target)

result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt

#### Create the EOP XML metadata file

In [ ]:
eop_metadata = dict()

eop_metadata['wkt'] = result_wkt
eop_metadata['startdate'] = parser.parse(min(dates)).strftime('%Y-%m-%dT%H:%M:%S')
eop_metadata['enddate'] = parser.parse(max(dates)).strftime('%Y-%m-%dT%H:%M:%S')

eop_xml = output_name + '.xml'
with open(eop_xml, 'wb') as file:
    file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
    file.write(eop_metadata)

#### Create the properties file for the reproducibility notebooks

In [ ]:
for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % output_name
    else: 
        title = 'Reproducibility stage-in notebook for Sentinel-1 data for generating %s' % output_name
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (search[0]['startdate'], search[0]['enddate']))
        file.write('geometry=%s' % (search[0]['wkt']))
        

### Compress the file

In [ ]:
with open(output_name + '.tif', 'rb') as f_in, gzip.open(output_name + '.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
    
os.remove(output_name + '.tif')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.